# Outline
1. Parse json to 
1. Do separate class_pcollection: asset1, asset2
1. split class_pcollections: train_asset1/2, eval_asset1/2
1. aggregate to TRAIN, TEST

In [ ]:
class mapquery(beam.DoFn):
    def process(self, element):
        key = element[0]
        value = element[1]
        yield key, value
#beam.GroupByKey()
# beam.combiners.Count.Globally() return number of rows

with beam.Pipeline() as p:
    res = (p | beam.io.ReadFromText("my_data.txt")
        | beam.Map(json.loads) | beam.Map(lambda x:some(**x)).with_output_types(some)
        | beam.ParDo(mapquery()) | beam.combiners.Count.PerKey()
          )
    res | beam.Map(print)

# | beam.ParDo(mapquery()) |beam.combiners.Count.PerKey()
# ('asset_1', 57)
# ('asset_2', 57)
# | beam.ParDo(mapquery()) | beam.combiners.Count.Globally()
# 114

# dataframe transfrom

In [1]:
#setup
import apache_beam as beam
import json
import typing
from apache_beam.dataframe.transforms import DataframeTransform
from apache_beam.dataframe.convert import to_dataframe
from apache_beam.dataframe.convert import to_pcollection

#set schema for a json element
class some(typing.NamedTuple):
    classes: str
    value: float

beam.coders.registry.register_coder(some, beam.coders.RowCoder)

#ouput folder
train_output_filename = "./result2/train/data.json"
eval_output_filename = "./result2/eval/data.json"

#splitting Pcollection
def split_dataset(elem, num_partitions, ratio):
    assert num_partitions == len(ratio)
    bucket = sum(map(ord, json.dumps(elem))) % sum(ratio)
    total = 0
    for i, part in enumerate(ratio):
        total += part
        if bucket < total:
            return i
    return len(ratio) - 1


#pipeline
with beam.Pipeline() as pdf:
    res = (pdf | "reading json file">>beam.io.ReadFromText("my_data.json")
        |"loading json">> beam.Map(json.loads) | beam.Map(lambda x:some(**x)).with_output_types(some))
    #converting to datafram
    df = to_dataframe(res)
    #creating df according to classes
    df1 = df.loc[df['classes']=='asset_1']
    df2 = df.loc[df['classes']=='asset_2']
    #Pcollections
    train_1,eval_1 = (to_pcollection(df1) | 'Partition1' >> beam.Partition(split_dataset, 2, ratio=[7, 3]))
    train_2,eval_2 = (to_pcollection(df2) | 'Partition2' >> beam.Partition(split_dataset, 2, ratio=[7, 3]))

    TRAIN = ((train_1,train_2) | "aggregate train">>beam.Flatten())
    EVAL = ((eval_1,eval_2)|"aggregate eval">> beam.Flatten())
    
    output = (TRAIN|"write to train">>beam.io.WriteToText(train_output_filename))
    output = (EVAL|"write to eval">>beam.io.WriteToText(eval_output_filename))

In [ ]:
train_output_filename = "./train/data.json"
eval_output_filename = "./eval/data.json"

def split_dataset(elem, num_partitions, ratio):
    assert num_partitions == len(ratio)
    bucket = sum(map(ord, json.dumps(elem))) % sum(ratio)
    total = 0
    for i, part in enumerate(ratio):
        total += part
        if bucket < total:
            return i
    return len(ratio) - 1


with beam.Pipeline() as pdf:
    res = (pdf | beam.io.ReadFromText("my_data.json")
        | beam.Map(json.loads) | beam.Map(lambda x:some(**x)).with_output_types(some))
    #just for one class
    df = to_dataframe(res)
    df1 = df.loc[df['classes']=='asset_1']
    df2 = df.loc[df['classes']=='asset_2']
    #Pcollections
    train_1,eval_1 = (to_pcollection(df1) | 'Partition1' >> beam.Partition(split_dataset, 2, ratio=[7, 3]))
    train_2,eval_2 = (to_pcollection(df2) | 'Partition2' >> beam.Partition(split_dataset, 2, ratio=[7, 3]))

    TRAIN = ((train_1,train_2) | "aggregate train">>beam.Flatten())
    EVAL = ((eval_1,eval_2)|"aggregate eval">> beam.Flatten())
    
    output = (TRAIN|"write to train">>beam.io.WriteToText(train_output_filename))
    output = (EVAL|"write to eval">>beam.io.WriteToText(eval_output_filename))